In [8]:
from env import config
import requests

In [9]:
MINDSDB_UN=config("MINDSDB_UN", default=None)# martyna@mindsdb.com
MINDSDB_PW=config("MINDSDB_PW", default=None) # xxxxxxxx
assert MINDSDB_UN is not None
assert MINDSDB_PW is not None


MINDSDB_BASE_URL = "https://cloud.mindsdb.com/api"

In [11]:
def get_mindsdb_session():
    session = requests.Session()
    session.post('https://cloud.mindsdb.com/cloud/login', json={
        'email': MINDSDB_UN,
        'password': MINDSDB_PW
    })
    return session

In [17]:
def mindsdb_query(session, sql_query):
    endpoint = "/sql/query"
    url = f"{MINDSDB_BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    return session.post(url, json={"query": sql_query}, headers=headers)

In [60]:

# ['flightDate',
#  'startingAirport',
#  'destinationAirport',
#  'isBasicEconomy',
#  'isRefundable',
#  'isNonStop',
#  'segmentsAirlineName',
#  'totalFare',
#  '__mdb_ts_previous_totalFare',
#  'select_data_query',
#  'when_data',
#  'totalFare_original',
#  'totalFare_confidence',
#  'totalFare_explain',
#  'totalFare_anomaly',
#  'totalFare_min',
#  'totalFare_max',
 
def predict_query(session, 
        flightDate = "2022-04-21", 
        startingAirport="SFO", 
        isNonStop=1, 
        destinationAirport="BOS",
        raw_request=True,
        ):
    sql_query = f"""
    SELECT m.flightDate, m.segmentsAirlineName, m.isNonStop, m.totalFare, m.totalFare_confidence FROM mindsdb.flight_price_predictor AS m
    JOIN ai_travel_agent.flight_prices AS t
    WHERE t.flightDate = "{flightDate}"
    AND t.startingAirport = "{startingAirport}"
    AND t.isNonStop = {isNonStop}
    AND t.destinationAirport = "{destinationAirport}";
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = response.json()
    columns = data.get("column_names")
    dataset = data.get('data')
    web_ready_data = [dict(zip(columns, row)) for row in dataset]
    return web_ready_data
        
    

In [61]:
session = get_mindsdb_session()
query_response = predict_query(session, 
        flightDate = "2022-04-21", 
        startingAirport="BOS", 
        isNonStop=1, 
        destinationAirport="SFO",
        raw_request=False,)
session.close()

In [62]:
query_response

[{'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'United',
  'isNonStop': 1.0,
  'totalFare': 437.70167433898956,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'American Airlines',
  'isNonStop': 1.0,
  'totalFare': 601.1875918728093,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'JetBlue Airways',
  'isNonStop': 1.0,
  'totalFare': 636.7351950883466,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'Alaska Airlines',
  'isNonStop': 1.0,
  'totalFare': 407.60097800046026,
  'totalFare_confidence': 0.9991},
 {'flightDate': '2022-04-21 00:00:00.000000',
  'segmentsAirlineName': 'Delta',
  'isNonStop': 1.0,
  'totalFare': 389.00500090014,
  'totalFare_confidence': 0.9991}]

In [40]:
# columns = data.get("column_names")
# dataset = data.get('data')
# columns

['segmentsAirlineName', 'isNonStop', 'totalFare', 'totalFare_confidence']

In [41]:
# dataset

[['Alaska Airlines', 1.0, 426.23859268457585, 0.9991],
 ['JetBlue Airways', 1.0, 354.99898339656704, 0.9991],
 ['American Airlines', 1.0, 404.39715167400135, 0.9991],
 ['Delta', 1.0, 352.59628080569775, 0.9991],
 ['United', 1.0, 347.99920213019533, 0.9991]]

In [44]:
# web_ready_data = []
# for row in dataset:
#     print(row)
#     for i, col in enumerate(row):
#         print(columns[i], col)

In [45]:
# web_ready_data = [dict(zip(columns, row)) for row in dataset]
# web_ready_data

[{'segmentsAirlineName': 'Alaska Airlines',
  'isNonStop': 1.0,
  'totalFare': 426.23859268457585,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'JetBlue Airways',
  'isNonStop': 1.0,
  'totalFare': 354.99898339656704,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'American Airlines',
  'isNonStop': 1.0,
  'totalFare': 404.39715167400135,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'Delta',
  'isNonStop': 1.0,
  'totalFare': 352.59628080569775,
  'totalFare_confidence': 0.9991},
 {'segmentsAirlineName': 'United',
  'isNonStop': 1.0,
  'totalFare': 347.99920213019533,
  'totalFare_confidence': 0.9991}]